In [1]:
import tensorflow as tf
import numpy  as np
import matplotlib.pyplot as plt
from PIL import Image

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
trX,trY = mnist.train.images,mnist.train.labels
teX,teY = mnist.test.images,mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [7]:
print('training images:',trX.shape)
print('testing images:',teX.shape)

training images: (55000, 784)
testing images: (10000, 784)


In [31]:
weights = tf.placeholder(dtype='float32',shape=[784,392])
bias_h = tf.placeholder(dtype='float32',shape=[392])
bias_v = tf.placeholder(dtype='float32',shape=[784])

In [32]:
X = tf.placeholder(shape=[None,784],dtype='float32', name='input')
mat = tf.add(tf.matmul(X,weights),bias_h,name='add_1')
h1 = tf.nn.sigmoid(mat)
h2 = tf.nn.relu(tf.sign(h1-tf.random_uniform(tf.shape(h1))))

In [15]:
with  tf.Session() as sess:
    a = tf.constant([0.7, 0.1, 0.8, 0.2])
    print (sess.run(a))
    b = sess.run(tf.random_uniform(tf.shape(a)))
    print (b)
    print (sess.run(a-b))
    print (sess.run(tf.sign( a - b)))
    print (sess.run(tf.nn.relu(tf.sign( a - b))))

[ 0.69999999  0.1         0.80000001  0.2       ]
[ 0.11043453  0.68030286  0.05827534  0.58409774]
[ 0.58956546 -0.58030283  0.74172467 -0.38409775]
[ 1. -1.  1. -1.]
[ 1.  0.  1.  0.]


In [35]:
mat1 = tf.add(tf.matmul(h2,tf.transpose(weights),name='matmul_2'),bias_v, name='add_2')
v1 = tf.nn.sigmoid(mat1)
v2 = tf.nn.relu(tf.sign(v1-tf.random_uniform(tf.shape(v1))))
h = tf.nn.sigmoid(tf.add(tf.matmul(v2,weights),bias_h))

In [36]:
alpha = 1.0
w_pos_grad = tf.matmul(tf.transpose(X),h2)
w_neg_grad = tf.matmul(tf.transpose(v2),h)
CD = (w_pos_grad-w_neg_grad)/tf.to_float(tf.shape(X)[0])
update_w = weights + alpha*CD
update_vb = bias_v + alpha*tf.reduce_mean(X-v2,0)
update_bb = bias_h + alpha*tf.reduce_mean(h2-h,0)

In [37]:
err = tf.reduce_mean(tf.square(X-v2))

In [38]:
cur_w = np.zeros([784,392],dtype='float32')
cur_h = np.zeros([392],dtype='float32')
cur_v = np.zeros([784],dtype='float32')
prv_w = np.zeros([784,392],dtype='float32')
prv_h = np.zeros([392],dtype='float32')
prv_v = np.zeros([784],dtype='float32')
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
sess.run(err,feed_dict={X:trX,weights:prv_w,bias_h:prv_h,bias_v:prv_v})        

0.4814975

In [41]:
epochs = 5
batch_size = 100
errors = []
for epoch in range(epochs):
    for start,end in zip(range(0,len(trX),batch_size),range(batch_size,len(trX),batch_size)):
        batch = trX[start:end]
        cur_w = sess.run(update_w,feed_dict={X:batch,weights:prv_w,bias_h:prv_h,bias_v:prv_v})
        cur_h = sess.run(update_bb,feed_dict={X:batch,weights:prv_w,bias_h:prv_h,bias_v:prv_v})
        cur_v = sess.run(update_vb,feed_dict={X:batch,weights:prv_w,bias_h:prv_h,bias_v:prv_v})
        prv_w = cur_w
        prv_h = cur_h
        prv_v = cur_v
        if start % 10000 ==0:
            errors.append(sess.run(err,feed_dict={X:batch,weights:prv_w,bias_h:prv_h,bias_v:prv_v}))
            
    print ('Epoch: %d' % epoch,'reconstruction error: %f' % errors[-1])

Epoch: 0 reconstruction error: 0.043107
Epoch: 1 reconstruction error: 0.043755
Epoch: 2 reconstruction error: 0.042841
Epoch: 3 reconstruction error: 0.043050
Epoch: 4 reconstruction error: 0.042879
